In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_431481/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 14:07:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 14:07:52 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Products table
products_data = {
    'product_id': [1, 1, 2, 2, 3, 3],
    'store': ['Shop', 'LC_Store', 'Nozama', 'Souq', 'Shop', 'Souq'],
    'price': [110, 100, 200, 190, 1000, 1900]
}

# Create DataFrame for Products
products_df = pd.DataFrame(products_data)

# Display the DataFrame
print("Products table:")
print(products_df)



df_person = spark.createDataFrame(products_df)
df_person.createOrReplaceTempView("Products")



Products table:
   product_id     store  price
0           1      Shop    110
1           1  LC_Store    100
2           2    Nozama    200
3           2      Souq    190
4           3      Shop   1000
5           3      Souq   1900


In [4]:
query = """
    select *
    from (
        select product_id, store, price from Products p
    )
    pivot (
        max(price) for store in (
            'LC_Store' LC_Store,
            'Nozama' Nozama,
            'Shop' Shop,
            'Souq' Souq 
        )
    ) order by product_id

"""

In [5]:
spark.sql(query).show()

+----------+--------+------+----+----+
|product_id|LC_Store|Nozama|Shop|Souq|
+----------+--------+------+----+----+
|         1|     100|  NULL| 110|NULL|
|         2|    NULL|   200|NULL| 190|
|         3|    NULL|  NULL|1000|1900|
+----------+--------+------+----+----+

